In [1]:
import numpy as np
import modes
import util
import spectra
from util import trapz, backtrapz, deriv
from matplotlib import pyplot as plt


In [2]:
# numerical parameters
NT = 4000
NK = 400
NX = 10000
L_MAX = 2500
xmin = 0.01
xmax = 3000
a0 = 1.e-6
a_rec = 1/1100

# arrays
A = np.linspace(a0, a_rec, NT)
K = np.linspace(.1, 3000, NK)
L = np.array([i for i in range(2, 50, 1)]+[i for i in range(50, 200, 5)] +
                  [i for i in range(200, L_MAX, 20)])
NL = len(L)
X = np.linspace(xmin, xmax, NX)
JLX, DJLX = spectra.get_bessels(L, X)

# physical parameters
h = 0.5
As = np.exp(3.064)/1.e10 # scalar perturbation amplitude
TCMB0 = 2.72548 * 1.e6 # CMB temp (microK)
OmegaM0 = 1.0
OmegaB0 = 0.05
OmegaR0 = 4.2e-5/h**2
OmegaN0 = OmegaR0*(1-1/1.68)
OmegaG0 = OmegaR0 - OmegaN0
OmegaC0 = OmegaM0 - OmegaB0
wC = np.zeros(NT)
wN = np.ones(NT)*1/3
cs2C = np.zeros((NT, NK))
cs2N = np.ones((NT, NK))*1/3

Bessel functions not found. Computing bessel functions...

Bessel functions stored.


In [3]:
"""Get the fiduciary parameters for the 2-fluid model"""

# solve for perturbation history using 3-fluid model.
Y, TAU = modes.solve_3fld(A, K, wC, cs2C, wN, cs2N, OmegaB0, OmegaC0,
           OmegaG0, OmegaN0)

Phi = Y[:, 0, :].T
deltaG = Y[:, 1, :].T
vG = Y[:, 2, :].T
deltaC =Y[:, 3, :].T
vC = Y[:, 4, :].T
deltaN = Y[:, 5, :].T
vN =Y[:, 6, :].T

OmegaB = OmegaB0 * A[::2]**-3
OmegaC = OmegaC0 * A[::2]**-3
OmegaN = OmegaN0 * A[::2]**-4
OmegaG = OmegaG0 * A[::2]**-4
OmegaD = OmegaN + OmegaC
OmegaD0 = OmegaC0 + OmegaN0

# calculate GDM functions
wD = OmegaN/(3*OmegaD)
deltaD = (OmegaC*deltaC + OmegaN*deltaN)/OmegaD
vD = (OmegaC*vC + 4/3*OmegaN*vN)/((1+wD)*OmegaD)
cs2D = (deltaN*OmegaN/3)/(deltaD*OmegaD)
delta_pD = deltaN

deltaD, vD, cs2D = deltaD.T, vD.T, cs2D.T

print(np.shape(deltaD.T[0]))

Y2, TAU2 = modes.solve_2fld(A[::2], K, wD, cs2D, deltaD[0], vD[0], OmegaB0,
               OmegaC0, OmegaG0, OmegaN0, OmegaD0, As, TCMB0, h)

plt.plot(A[::2],Y[:,0,::50])
plt.plot(A[::4],Y2[:,0,::50],linestyle=':',linewidth=3)
plt.show()

(2000,)


TypeError: too many arguments: expected 11, got 14